In [ ]:
!pip install --no-deps -r https://raw.githubusercontent.com/Midnighter/combine2020-equilibrator/picp/colab_reversibility.txt

# Equilibrator for Metabolic Network Analysis: Thermodynamic Profiling and Enzyme-Cost Minimization

Moritz E. Beber & Elad Noor

## Fundamentals

The direction of net flux through a reaction $r$ is given by the sign of its transformed Gibbs free energy at physiological conditions $ \operatorname{sgn} \left( \Delta_r G' \right) $.

\begin{align}
\Delta_r G' & = \Delta_r G'^\circ + R T \cdot \ln \left( Q_r \right) \\
\ln \left( Q_r \right) & = \mathbf{ S }_r^{ \top } \mathbf{ x }
\end{align}

* $ \Delta_r G'^\circ $: standard transformed Gibbs energy of reaction $ r $ (assumes $ 1 ~ \mathrm{mol} \mspace{3mu} \mathrm{L}^{-1} $ concentrations)
* $ R $: gas constant
* $ T $: temperature
* $ Q_r $: reaction quotient
* $ \mathbf{ S }_r $: column vector of the stoichiometric matrix, representing reaction $ r $
* $ \mathbf{ x } $: vector of natural logarithm of metabolite concentrations

\begin{align}
\Delta_r G' & = \Delta_r G'^\circ + R T \cdot \mathbf{ S }_r^{ \top } \mathbf{ x } \\
\end{align}

* $ \Delta_r G' < 0 $: net flux proceeds in forward direction
* $ \Delta_r G' > 0 $: net flux proceeds in reverse direction
* $ \Delta_r G' = 0 $: reaction is in equilibrium

$ -\Delta_r G' $ is also called the _driving force_

## How do we obtain the standard transformed Gibbs energy of a reaction?

* Measure it $ \Delta_r G'^\circ = - R T \ln \left( K'_r \right) $ at apparent equilibrium
* Measure energy of compound formation $ \Delta_r G'^\circ \approx \mathbf{ S }_r^{ \top } \Delta_f G^\circ $ known as Reactant Contribution (RC) uses reference points
* Group Contribution (GC) $ \Delta_r G'^\circ \approx \mathbf{ S }_r^{ \top } \mathbf{G} \Delta_g G^\circ $
* Component Contribution (CC) combines RC and GC

1. Jankowski, Matthew D., Christopher S. Henry, Linda J. Broadbelt, and Vassily Hatzimanikatis. 2008. “Group Contribution Method for Thermodynamic Analysis of Complex Metabolic Networks.” Biophysical Journal 95 (3): 1487–99. https://doi.org/10.1529/biophysj.107.124784.
2. Noor, Elad, Arren Bar-Even, Avi Flamholz, Yaniv Lubling, Dan Davidi, and Ron Milo. 2012. “An Integrated Open Framework for Thermodynamics of Reactions That Combines Accuracy and Coverage.” Bioinformatics 28 (15): 2037–44. https://doi.org/10.1093/bioinformatics/bts317.
3. Noor, Elad, Hulda S. Haraldsdóttir, Ron Milo, and Ronan M. T. Fleming. 2013. “Consistent Estimation of Gibbs Energy Using Component Contributions.” PLOS Computational Biology 9 (7): e1003098. https://doi.org/10.1371/journal.pcbi.1003098.
4. Du, Bin, Zhen Zhang, Sharon Grubner, James T. Yurkovich, Bernhard O. Palsson, and Daniel C. Zielinski. 2018. “Temperature-Dependent Estimation of Gibbs Energies Using an Updated Group-Contribution Method.” Biophysical Journal 114 (11): 2691–2702. https://doi.org/10.1016/j.bpj.2018.04.030.


## eQuilibrator

## Database Content

<img src="../images/metanetx-4.0.png" style="height: 15cm; display: inline-block;" />

In [ ]:
from equilibrator_api import ComponentContribution
from equilibrator_cache.models import Compound

In [ ]:
cc = ComponentContribution()

In [ ]:
print(f"{cc.ccache.session.query(Compound).count():,d}")

In [ ]:
structures = [
    c.inchi_key[:-2]
    for c in cc.ccache.session.query(Compound.inchi_key).filter(Compound.inchi_key.isnot(None))
]
print(f"{len(set(structures)):,d}")

## What can we use the transformed Gibbs energies for?

* [Max-min driving force](https://gitlab.com/equilibrator/equilibrator-pathway/-/blob/develop/examples/ecoli_noor_2016.ipynb)
* Reversibility index
* TMFA
* TMFA sampling

### Max-min driving force (MDF)

\begin{eqnarray}
    \text{maximize} & B \\
    \text{subject to} &  -\Delta_r \mathbf{G}' & \geq B \\
    & \Delta_r \mathbf{G}' &= \Delta_r \mathbf{G}'^\circ + RT \cdot \mathbf{S}^\top \cdot \mathbf{x} \\
    & \ln(\mathbf{c}_{\min}) &\leq \mathbf{x} \leq \ln(\mathbf{c}_{\max})
\end{eqnarray}

### Reversibility Index

* Reversibility index $ \gamma $ represents the required change in concentration to reverse reaction direction
* $ \gamma = 1000 \equiv \ln ( \gamma ) \approx 7 $ represents our threshold of three orders of magnitude


Noor, Elad, Arren Bar-Even, Avi Flamholz, Yaniv Lubling, Dan Davidi, and Ron Milo. 2012. “An Integrated Open Framework for Thermodynamics of Reactions That Combines Accuracy and Coverage.” Bioinformatics 28 (15): 2037–44. https://doi.org/10.1093/bioinformatics/bts317.

In [ ]:
from operator import attrgetter
from cobra.io import load_model
from equilibrator_cache import CompoundCache
from equilibrator_api.compatibility import map_cobra_reactions

In [ ]:
model = load_model("e_coli_core")

In [ ]:
reactions = sorted(set(model.reactions) - set(model.boundary), key=attrgetter("id"))
len(reactions)

In [ ]:
mapping = map_cobra_reactions(cache=cc.ccache, reactions=reactions)

In [ ]:
len(mapping)

In [ ]:
for rxn_id, eq_rxn in mapping.items():
    ln_gamma = cc.ln_reversibility_index(eq_rxn)
    rxn = model.reactions.get_by_id(rxn_id)
    if rxn.reversibility and abs(ln_gamma) > 7.0:
        print(rxn_id, "is reversible but ln[gamma] =", ln_gamma)
    elif not rxn.reversibility and abs(ln_gamma) < 7.0:
        print(rxn_id, "is irreversible but ln[gamma] =", ln_gamma)

In [ ]:
model.reactions.THD2

## Thank You & Discussion

Please reach out to us with questions, suggestions, or problems.

* Chat with us on <a href="https://gitter.im/equilibrator-devs/equilibrator-api">gitter.im</a>
* Open an issue on https://gitlab.com/equilibrator/equilibrator-api
* <a href="mailto:noor@imsb.biol.ethz.ch?subject=equilibrator"><img src="../images/envelope-solid.svg" alt="email" style="height: 1.2em; display: inline-block;" /> eladnoor@protonmail.com</a>
* <a href="mailto:moritz@unseenbio.com?subject=equilibrator"><img src="../images/envelope-solid.svg" alt="email" style="height: 1.2em; display: inline-block;" /> moritz.beber@posteo.de</a>